# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [33]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

import panel as pn
from panel.interact import interact

In [2]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [3]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

coord_path = Path("Data/neighborhoods_coordinates.csv")
coord_data = pd.read_csv(coord_path)
coord_data.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [64]:
# Define Panel Visualization Functions
def housing_units_per_year():    
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    #set up data
    sfo_house_units = pd.DataFrame(sfo_data.groupby('year')['housing_units'].mean())
    tot_housing = sfo_data.groupby(['year'])['housing_units'].mean()
    #plot
    sfo_house_units.plot.bar(title = "Housing Units in San Francisco from 2010 to 2016",  ylim = (tot_housing.min() - tot_housing.std(), tot_housing.max() + tot_housing.std()))



def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    # awr up data
    sfo_house_units = pd.DataFrame(sfo_data.groupby('year')['housing_units'].mean())
    sfo_house_units['sale_price_sqr_foot'] = sfo_data.groupby('year')['sale_price_sqr_foot'].mean()
    sfo_house_units ['gross_rent'] = sfo_data.groupby('year')['gross_rent'].mean()
    #plot
    fig2, ax2 = plt.subplots()
    ax2.plot(sfo_house_units.index, sfo_house_units['gross_rent'],color='tab:orange' )
    ax2.set_ylabel('Price per SqFt', fontweight="bold", fontsize = 12)
    ax2.set_xlabel('Year',  fontweight="bold", fontsize = 12)
    ax2.set_title('Average Gross Rent by Year',fontweight="bold", fontsize = 12 )



def average_sales_price(neighborhood):
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    #set up data
    sfo_house_units = pd.DataFrame(sfo_data.groupby('year')['housing_units'].mean())
    sfo_house_units['sale_price_sqr_foot'] = sfo_data.groupby('year')['sale_price_sqr_foot'].mean()
    sfo_house_units ['gross_rent'] = sfo_data.groupby('year')['gross_rent'].mean()
    #plot
    fig1, ax1 = plt.subplots()
    ax1.plot(sfo_house_units.index, sfo_house_units['sale_price_sqr_foot'], color='tab:blue')
    ax1.set_ylabel('Price per SqFt', fontweight="bold", fontsize = 12)
    ax1.set_xlabel('Year',  fontweight="bold", fontsize = 12)
    ax1.set_title('Average Price per SqFt by Year',fontweight="bold", fontsize = 12 )


def average_price_by_neighborhood(neighborhood):
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    # set up data
    sfo_neighb = pd.DataFrame(columns = ['sale_price_sqr_foot', 'housing_units', 'gross_rent'])
    sfo_neighb['sale_price_sqr_foot'] = sfo_data.groupby(['year','neighborhood' ])['sale_price_sqr_foot'].mean()
    sfo_neighb ['gross_rent'] = sfo_data.groupby(['year','neighborhood' ])['gross_rent'].mean()
    sfo_neighb ['housing_units'] = sfo_data.groupby(['year','neighborhood' ])['housing_units'].mean()
    sfo_neighb.reset_index(inplace = True)
    
    #set up plotting datafraem
    disp_df = pd.DataFrame()
    disp_df['years'] = sfo_neighb.loc[sfo_neighb['neighborhood'] == neighborhood, 'year']
    disp_df['sale_price_sqr_foot'] = sfo_neighb.loc[sfo_neighb['neighborhood'] == neighborhood, 'sale_price_sqr_foot']
    
    #return plot
    return disp_df.hvplot.line(
        x = 'years',
        y = 'sale_price_sqr_foot',
        title = (f'Neighborhood:  {neighborhood}'),

    )
    

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    # need to debug
    # YOUR CODE HERE!
    # set up data in two steps from rental analysis. may be combined into one.
#     print ('test 1')
    sfo_neighb = pd.DataFrame(columns = ['sale_price_sqr_foot', 'housing_units', 'gross_rent'])
#     print ('test 1')
    sfo_neighb['sale_price_sqr_foot'] = sfo_data.groupby(['year','neighborhood' ])['sale_price_sqr_foot'].mean()
    sfo_neighb ['gross_rent'] = sfo_data.groupby(['year','neighborhood' ])['gross_rent'].mean()
    sfo_neighb ['housing_units'] = sfo_data.groupby(['year','neighborhood' ])['housing_units'].mean()
    print (f'debug sfo_neigh: {sfo_neighb.head(5)}')
    sfo_avg_df = pd.DataFrame()
    sfo_avg_df ['sale_price_sqr_foot'] = sfo_neighb.groupby('neighborhood')['sale_price_sqr_foot'].mean()
    sfo_avg_df ['housing_units'] = sfo_neighb.groupby('neighborhood')['housing_units'].mean()
    sfo_avg_df ['gross_rent'] = sfo_neighb.groupby('neighborhood')['gross_rent'].mean()
    sf_topten_df = sfo_avg_df.nlargest(10,'gross_rent' )
    sf_topten_df.sort_values(by = 'sale_price_sqr_foot', inplace = True)
    sf_topten_df.reset_index(inplace = True)
    print (f'debug sf_topten_df: {sf_topten_df.head(5)}')
#     sf_topten_df.index
    #plot
    print (f'debug sf_topten_df[neighborhood]: { sf_topten_df["neighborhood"]}')
#     print (f'debug sf_topten_df[sale_price_sqr_foot]: { sf_topten_df["sale_price_sqr_foot"]}')
    sf_topten_df.hvplot.bar(
        x = 'neighborhood',
        y = 'sale_price_sqr_foot',
        title = ('Top 10 Most Expensive Neighborhoods'),
        rot = 90
    )
    

def most_expensive_neighborhoods_rent_sales(Neighbourhood):
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
    #prepare summary data
    test_df = pd.DataFrame()
    test_df['gross_rent'] = sfo_neighb.groupby(['year', 'neighborhood'])['gross_rent'].mean()
    test_df['sale_price_sqr_foot']= sfo_neighb.groupby(['year', 'neighborhood'])['sale_price_sqr_foot'].mean()
    test_df.reset_index(inplace=True)

    #prepare data for display
    disp_df = pd.DataFrame()
    disp_df['years'] = test_df.loc[test_df['neighborhood'] == Neighbourhood, 'year']
    disp_df['gross_rent'] = test_df.loc[test_df['neighborhood'] == Neighbourhood, 'gross_rent']
    disp_df['sale_price_sqr_foot'] = test_df.loc[test_df['neighborhood'] == Neighbourhood, 'sale_price_sqr_foot']


    return(disp_df.hvplot.bar(
        x = 'years',
        y = ['gross_rent', 'sale_price_sqr_foot' ],
        value_label = "Price",
        rot = 90,
        title = (f'Top 10 Expensive Neighborhoods: {Neighbourhood}'))
    )
  
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!



def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!


In [66]:
    sfo_neighb = pd.DataFrame(columns = ['sale_price_sqr_foot', 'housing_units', 'gross_rent'])
#     print ('test 1')
    sfo_neighb['sale_price_sqr_foot'] = sfo_data.groupby(['year','neighborhood' ])['sale_price_sqr_foot'].mean()
    sfo_neighb ['gross_rent'] = sfo_data.groupby(['year','neighborhood' ])['gross_rent'].mean()
    sfo_neighb ['housing_units'] = sfo_data.groupby(['year','neighborhood' ])['housing_units'].mean()
#     print (f'debug sfo_neigh: {sfo_neighb.head(5)}')
    sfo_avg_df = pd.DataFrame()
    sfo_avg_df ['sale_price_sqr_foot'] = sfo_neighb.groupby('neighborhood')['sale_price_sqr_foot'].mean()
    sfo_avg_df ['housing_units'] = sfo_neighb.groupby('neighborhood')['housing_units'].mean()
    sfo_avg_df ['gross_rent'] = sfo_neighb.groupby('neighborhood')['gross_rent'].mean()
    sf_topten_df = sfo_avg_df.nlargest(10,'gross_rent' )
    sf_topten_df.sort_values(by = 'sale_price_sqr_foot', inplace = True)
    sf_topten_df.reset_index(inplace = True)
#     print (f'debug sf_topten_df: {sf_topten_df.head(5)}')
    sf_topten_df.index
    #plot
#     print (f'debug sf_topten_df[neighborhood]: { sf_topten_df["neighborhood"]}')
#     print (f'debug sf_topten_df[sale_price_sqr_foot]: { sf_topten_df["sale_price_sqr_foot"]}')
    sf_topten_df.hvplot.bar(
        x = 'neighborhood',
        y = 'sale_price_sqr_foot',
        title = ('Top 10 Most Expensive Neighborhoods'),
        rot = 90
    )

:Bars   [neighborhood]   (sale_price_sqr_foot)

In [67]:
top_most_expensive_neighborhoods()

debug sfo_neigh:                        sale_price_sqr_foot  housing_units  gross_rent
year neighborhood                                                    
2010 Alamo Square               291.182945         372560        1239
     Anza Vista                 267.932583         372560        1239
     Bayview                    170.098665         372560        1239
     Buena Vista Park           347.394919         372560        1239
     Central Richmond           319.027623         372560        1239
debug sf_topten_df:         neighborhood  sale_price_sqr_foot  housing_units   gross_rent
0     Silver Terrace           170.292549      380348.00  3528.000000
1  Visitacion Valley           301.466180      381321.50  3657.000000
2         Anza Vista           373.382198      379050.00  3031.833333
3          Excelsior           388.765927      379050.00  3031.833333
4     Inner Parkside           519.385604      379861.25  3224.000000
debug sf_topten_df[neighborhood]: 0       Silver Terr

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!


# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [65]:
# top_most_expensive_neighborhoods()

debug sfo_neigh:                        sale_price_sqr_foot  housing_units  gross_rent
year neighborhood                                                    
2010 Alamo Square               291.182945         372560        1239
     Anza Vista                 267.932583         372560        1239
     Bayview                    170.098665         372560        1239
     Buena Vista Park           347.394919         372560        1239
     Central Richmond           319.027623         372560        1239
debug sf_topten_df:         neighborhood  sale_price_sqr_foot  housing_units   gross_rent
0     Silver Terrace           170.292549      380348.00  3528.000000
1  Visitacion Valley           301.466180      381321.50  3657.000000
2         Anza Vista           373.382198      379050.00  3031.833333
3          Excelsior           388.765927      379050.00  3031.833333
4     Inner Parkside           519.385604      379861.25  3224.000000
debug sf_topten_df[neighborhood]: 0       Silver Terr

In [68]:
most_expensive_neighborhoods_rent_sales()

NameError: name 'Neighbourhood' is not defined

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()